# 머신러닝 회귀 모델

In [1]:
# 라이브러리와 데이터 로드

import os
import pandas as pd 
import numpy as np 

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.types import *
import pyspark.sql.functions as F 
from pyspark.sql.functions import udf, col 

from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml.feature import VectorAssembler, StandardScaler

import seaborn as sns 
import matplotlib.pyplot as plt 

# Setting for visualization
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', 400)

from matplotlib import rcParams
sns.set(context='notebook', style='whitegrid',
        rc={'figure.figsize': (18, 4)})
rcParams['figure.figsize'] = 18, 4

# 노트북 재실행을 대비하기 위해 랜덤 시드 설정
rnd_seed = 23
np.random.seed = rnd_seed
np.random.set_state = rnd_seed

In [2]:
data_path = './data/cal_housing.data'
# data의 컬럼명 스키마로 명시
schema_string = 'long lat medage totrooms totbdrms pop houshlds medinc medhv'

fields = [StructField(field, FloatType(), True) for field in schema_string.split()]
schema = StructType(fields)

# 데이터 로드를 위한 SparkSession 만들기
spark = SparkSession.builder.master('local[2]')\
        .appName('Linear-Regression-California-housing')\
        .getOrCreate()

# 데이터 로드
housing_df = spark.read.csv(path=data_path, schema=schema).cache()

# take()는 데이터 미리보기를 Python named tuple 형태인 DataFrame의 Row 형식으로 보여준다.
housing_df.take(5)

[Row(long=-122.2300033569336, lat=37.880001068115234, medage=41.0, totrooms=880.0, totbdrms=129.0, pop=322.0, houshlds=126.0, medinc=8.325200080871582, medhv=452600.0),
 Row(long=-122.22000122070312, lat=37.86000061035156, medage=21.0, totrooms=7099.0, totbdrms=1106.0, pop=2401.0, houshlds=1138.0, medinc=8.301400184631348, medhv=358500.0),
 Row(long=-122.23999786376953, lat=37.849998474121094, medage=52.0, totrooms=1467.0, totbdrms=190.0, pop=496.0, houshlds=177.0, medinc=7.257400035858154, medhv=352100.0),
 Row(long=-122.25, lat=37.849998474121094, medage=52.0, totrooms=1274.0, totbdrms=235.0, pop=558.0, houshlds=219.0, medinc=5.643099784851074, medhv=341300.0),
 Row(long=-122.25, lat=37.849998474121094, medage=52.0, totrooms=1627.0, totbdrms=280.0, pop=565.0, houshlds=259.0, medinc=3.8461999893188477, medhv=342200.0)]

# 데이터 탐색

In [3]:
# 그룹핑해서 집계
result_df = housing_df.groupBy('medage').count()
# 값 정렬 기준 설정해주고 내림차순으로 정렬
result_df.sort('medage', ascending=False).show(5)

# 수치형 변수들 기술통계량으로 살펴보기
housing_df.describe().show(10)

+------+-----+
|medage|count|
+------+-----+
|  52.0| 1273|
|  51.0|   48|
|  50.0|  136|
|  49.0|  134|
|  48.0|  177|
+------+-----+
only showing top 5 rows

+-------+-------------------+-----------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+
|summary|               long|              lat|            medage|          totrooms|         totbdrms|               pop|         houshlds|            medinc|             medhv|
+-------+-------------------+-----------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+
|  count|              20640|            20640|             20640|             20640|            20640|             20640|            20640|             20640|             20640|
|   mean|-119.56970444871473|35.63186143109965|28.639486434108527|2635.7630813953488|537.8980135658915|1425.4767441860465|49

In [4]:
# 반환되는 통계량 값들을 소수점 제거하거나 하는 등 커스터마이징해서 출력
(housing_df.describe()).select('summary',
                              F.round('medage', 4).alias('medage'),
                              F.round('totrooms', 4).alias('totrooms'),
                              F.round('totbdrms', 4).alias('totbdrms'),
                              F.round('pop', 4).alias('pop'),
                              F.round('houshlds', 4).alias('houshlds'),
                              F.round('medinc', 4).alias('medinc'),
                              F.round('medhv', 4).alias('medhv')).show(10)

+-------+-------+---------+--------+---------+--------+-------+-----------+
|summary| medage| totrooms|totbdrms|      pop|houshlds| medinc|      medhv|
+-------+-------+---------+--------+---------+--------+-------+-----------+
|  count|20640.0|  20640.0| 20640.0|  20640.0| 20640.0|20640.0|    20640.0|
|   mean|28.6395|2635.7631| 537.898|1425.4767|499.5397| 3.8707|206855.8169|
| stddev|12.5856|2181.6153|421.2479|1132.4621|382.3298| 1.8998|115395.6159|
|    min|    1.0|      2.0|     1.0|      3.0|     1.0| 0.4999|    14999.0|
|    max|   52.0|  39320.0|  6445.0|  35682.0|  6082.0|15.0001|   500001.0|
+-------+-------+---------+--------+---------+--------+-------+-----------+



# Feature Engineering

In [5]:
# 파생변수 생성
housing_df = (housing_df.withColumn('rmsperhh',
                                    F.round(col('totrooms')/col('houshlds'), 2))\
                        .withColumn('popperhh',
                                    F.round(col('pop')/col('houshlds'), 2))\
                        .withColumn('bdrmsperrm',
                                    F.round(col('totbdrms')/col('totrooms'), 2)))

housing_df.show(5)

+-------+-----+------+--------+--------+------+--------+------+--------+--------+--------+----------+
|   long|  lat|medage|totrooms|totbdrms|   pop|houshlds|medinc|   medhv|rmsperhh|popperhh|bdrmsperrm|
+-------+-----+------+--------+--------+------+--------+------+--------+--------+--------+----------+
|-122.23|37.88|  41.0|   880.0|   129.0| 322.0|   126.0|8.3252|452600.0|    6.98|    2.56|      0.15|
|-122.22|37.86|  21.0|  7099.0|  1106.0|2401.0|  1138.0|8.3014|358500.0|    6.24|    2.11|      0.16|
|-122.24|37.85|  52.0|  1467.0|   190.0| 496.0|   177.0|7.2574|352100.0|    8.29|     2.8|      0.13|
|-122.25|37.85|  52.0|  1274.0|   235.0| 558.0|   219.0|5.6431|341300.0|    5.82|    2.55|      0.18|
|-122.25|37.85|  52.0|  1627.0|   280.0| 565.0|   259.0|3.8462|342200.0|    6.28|    2.18|      0.17|
+-------+-----+------+--------+--------+------+--------+------+--------+--------+--------+----------+
only showing top 5 rows



이제 필요한 변수들만 골라서 데이터프레임을 새로 할당한 후 수치형 변수들에 Scaling을 적용시켜보자. 특이한 점은 Scaling을 적용시키기 이전에 필요한 Feature들을 Vector로 변환하는 작업이 선행되어야 한다는 것이다.

In [6]:
# 사용하지 않을 변수 제외하고 필요한 변수들만 select
housing_df = housing_df.select('medhv',
                              'totbdrms',
                              'pop',
                              'houshlds',
                              'medinc',
                              'rmsperhh',
                              'popperhh',
                              'bdrmsperrm')

featureCols = ['totbdrms', 'pop', 'houshlds', 'medinc',
                'rmsperhh', 'popperhh', 'bdrmsperrm']

# VectorAssembler로 feature vector로 변환
assembler = VectorAssembler(inputCols=featureCols, outputCol='features')
assembled_df = assembler.transform(housing_df)
assembled_df.show(10, truncate=True)

+--------+--------+------+--------+------+--------+--------+----------+--------------------+
|   medhv|totbdrms|   pop|houshlds|medinc|rmsperhh|popperhh|bdrmsperrm|            features|
+--------+--------+------+--------+------+--------+--------+----------+--------------------+
|452600.0|   129.0| 322.0|   126.0|8.3252|    6.98|    2.56|      0.15|[129.0,322.0,126....|
|358500.0|  1106.0|2401.0|  1138.0|8.3014|    6.24|    2.11|      0.16|[1106.0,2401.0,11...|
|352100.0|   190.0| 496.0|   177.0|7.2574|    8.29|     2.8|      0.13|[190.0,496.0,177....|
|341300.0|   235.0| 558.0|   219.0|5.6431|    5.82|    2.55|      0.18|[235.0,558.0,219....|
|342200.0|   280.0| 565.0|   259.0|3.8462|    6.28|    2.18|      0.17|[280.0,565.0,259....|
|269700.0|   213.0| 413.0|   193.0|4.0368|    4.76|    2.14|      0.23|[213.0,413.0,193....|
|299200.0|   489.0|1094.0|   514.0|3.6591|    4.93|    2.13|      0.19|[489.0,1094.0,514...|
|241400.0|   687.0|1157.0|   647.0|  3.12|     4.8|    1.79|      0.22

Scaling 하는 API도 PySpark에서 제공하는데 여러가지 스케일링 방법이 있지만 여기서는 표준화인 StandardScaler를 사용해보자. 메소드 사용 방법은 다음과 같다. StandardScaler(inputCol='Raw Features', outputCol='Scaled Features') 로 스케일링을 정의해주고 fit(dataframe)과 transform(dataframe)으로 스케일링을 수행해주자.

In [7]:
# Feature들을 스케일링 시키기

standdardScaler = StandardScaler(inputCol='features', outputCol='features_scaled')

scaled_df = standdardScaler.fit(assembled_df).transform(assembled_df)

# scaling된 피쳐들만 추출해보기
scaled_df.select('features', 'features_scaled').show(10, truncate=True)

+--------------------+--------------------+
|            features|     features_scaled|
+--------------------+--------------------+
|[129.0,322.0,126....|[0.30623297630686...|
|[1106.0,2401.0,11...|[2.62553233949916...|
|[190.0,496.0,177....|[0.45104081781631...|
|[235.0,558.0,219....|[0.55786627466754...|
|[280.0,565.0,259....|[0.66469173151877...|
|[213.0,413.0,193....|[0.50564049576249...|
|[489.0,1094.0,514...|[1.16083663111672...|
|[687.0,1157.0,647...|[1.63086864126214...|
|[665.0,1206.0,595...|[1.57864286235709...|
|[707.0,1551.0,714...|[1.67834662208491...|
+--------------------+--------------------+
only showing top 10 rows



# 데이터 분할과 회귀 모델링

회귀 모델은 여러가지가 있지만 여기서는 오버피팅을 예방하기 위한 정규화 항이 포함되어 있는 Elastic Net 회귀 모델사용해보자

In [8]:
# data split
train_data, test_data = scaled_df.randomSplit([0.8, 0.2], seed=rnd_seed)

# Elastic Net
lr = LinearRegression(featuresCol='features_scaled',
                      labelCol='medhv',
                      predictionCol='predmedhv',
                      maxIter=10,
                      regParam=0.3,
                      elasticNetParam=0.8,
                      standardization=False)

# model training
linearModel = lr.fit(train_data)

# 테스트 데이터에 대한 예측

In [9]:
predictions = linearModel.transform(test_data)

print(type(predictions))

<class 'pyspark.sql.dataframe.DataFrame'>


In [10]:
# y값과 예측값만 추출해서 비교해보기
pred_labels = predictions.select('predmedhv', 'medhv')
pred_labels.show()

+------------------+-------+
|         predmedhv|  medhv|
+------------------+-------+
|  65820.1760042106|14999.0|
|221374.23211624727|22500.0|
| 40972.55542160492|22500.0|
|111770.06878754942|26900.0|
| 176224.2459104087|34400.0|
| 80414.73100645898|36700.0|
|145622.16563780906|37500.0|
| 78208.77719512783|39400.0|
|136096.33496158314|39800.0|
|108725.69135250017|40900.0|
| 76631.25587698547|41700.0|
| 75815.91817725686|42500.0|
|101987.67841479904|42500.0|
| 80076.98310851591|43000.0|
| 76251.18681418494|43600.0|
|108256.88381314362|44000.0|
| 93202.79491312054|44000.0|
| 92720.30368070511|44400.0|
| 66433.07339323626|44500.0|
|141201.75137895203|44600.0|
+------------------+-------+
only showing top 20 rows



In [11]:
# 그렇다면 예측값과 실제값이 얼마나 차이나는지 RMSE, MAE를 계산해보고 만들어진 회귀식이 주어진 데이터를 얼마나 잘 설명하는지 나타내는 R2 Score(결정계수)값을 살펴보자.

print(f"RMSE:{linearModel.summary.rootMeanSquaredError}")
print(f"MAE: {linearModel.summary.meanAbsoluteError}")
print(f"R2 score: {linearModel.summary.r2}")

RMSE:78116.2564798975
MAE: 56925.44456037988
R2 score: 0.5435224404301482


추가적으로 메트릭을 얻을 수 있는 방법은 위 코드 말고도 RegressionEvaluator로도 계산할 수 있다. 해당 메소드의 사용방법은 다음과 같다.

- RegressionEvaluator(predictionCol, labelCol, metricName)
    - predictionCol : 예측값
    - labelCol : 실제값
    - metricName : 측정할 메트릭 이름(ex. RMSE, MAE 등등)

In [13]:
# RMSE
evaluator = RegressionEvaluator(predictionCol='predmedhv',
                               labelCol='medhv',
                               metricName='rmse')
# MAE                            
evaluator = RegressionEvaluator(predictionCol='predmedhv',
                               labelCol='medhv',
                               metricName='mae')
# R2 Score
evaluator = RegressionEvaluator(predictionCol='predmedhv',
                               labelCol='medhv',
                               metricName='r2')

print(f"RMSE: {evaluator.evaluate(pred_labels)}")
print(f"MAE: {evaluator.evaluate(pred_labels)}")  
print(f"R2 score: {evaluator.evaluate(pred_labels)}")


RMSE: 0.5595706743501381
MAE: 0.5595706743501381
R2 score: 0.5595706743501381


In [14]:
# 또 다른 방법은 RegressionMetrics 메소드인데 이는 특이하게 예측값을 RDD 자료구조로 변환하고 넣어주어야 한다.
# 예측값인 pred_labels를 RDD 자료구조로 변환하고 넣어야 함
metrics = RegressionMetrics(pred_labels.rdd)

print("RMSE:", metrics.rootMeanSquaredError)
print("MAE:", metrics.meanAbsoluteError)
print("R2 score:", metrics.r2)

RMSE: 75954.86368997344
MAE: 55516.495481755104
R2 score: 0.5595706743501381
